# Web Scraping multiple pages 

We have practiced web scraping when all the information we wanted was on a single table of a site. What happens when we want to scrape information from multiple pages?

## First example - IMDB 

Go to https://www.imdb.com/search/title/ and enter the following parameters, leaving all other fields blank or with its default value:

- Title Type: Feature film

- Release date: From 1990 to 1992

- User Rating: 7.5 to "-"

The page you get should be familiar. There's a list with movies and each movie has its title, release year, crew, etc. You could inspect the page and build the code to collect the date.

Note the resulting query obtained contain hundreds of movies, and each page only contains 50 of them (you can change the settings to obtain up to 250 movies/page, but that still won't be the complete list).

One way to automatize multi page web scraping is to look at the URLs. 

https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,

Note what the url looks like if you scroll down and click on "Next", the URL is now: 

https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt

Can you see the pattern?

our search options are in the parameters title_type, release_date and user_rating. Then, we have the start parameter, which jumps in intervals of 50, and the ref_ parameter, which takes the value of "adv_nxt".

In [ ]:
#  import libraries
from bs4 import BeautifulSoup
import requests

In [ ]:
#  url: this time, start with the 'second' page
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt"

In [ ]:
# download html with a request, check response code 
response=requests.get(url)
response.status_code

In [ ]:
#  parse html (create the 'soup')
soup=BeautifulSoup(response.content, "html.parser")

# check that the html code looks as expected 


Now, we'll have to build a list of values which jumps by 50, up to the total number of movies we want to scrape.  

In [ ]:
# define iterations 




In [ ]:
# check the iterations work




In [ ]:
# create the url string for the page search, populate with the iterations




In [ ]:
# test the urls 



### Respectful scraping:

Before starting with the actual scraping, though, there's something we need to note when sending automated requests to websites: it's good practice to let a few seconds pass in between requests. 

Some pages don't like being scraped and will block your IP if they detect you are sending automated requests. Others might have a small server for the traffic they handle, and sending too many requests might crash the site.

The sleep module will help us with that. 

In [ ]:
from time import sleep




In [ ]:
# To make it more "human", we can randomize the waiting time:



### Assembling the script to send and store multiple requests

ingredients for our multi page scraper : 
    + iterations 
    + url list with iterations 
    + sleepy time + random gaps (to look human)

In [ ]:
pages =[]
#assemble urls
for i in iterations:
    start_at =str(i)
    url="https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=" + start_at + "&ref_=adv_nxt"
#download html with get request
    response = requests.get(url)
#monitor the status codes for each page 
    print("status=" +str(response.status_code))
#store pages into a list 
    pages.append(response)
#respectful nap time 
    wait_time=randint(1,4)
    print("i will sleep for..." +str(wait_time) +" seconds now")
    sleep(wait_time)


In [ ]:
BeautifulSoup(pages[0].content,"html.parser")

Note: if you print the object pages after running the code above, you'll just see the response code messages, but the html code is still accessible and you can parse it the same way as before

### Build code to collect the relevant information from the Request 

this is what we need : 

##### Parse just the first page, for testing purposes
- soup=BeautifulSoup(pages[0].content, "html.parser")

##### title and synopsis

- soup.select("div.lister-item-content > h3 > a")
- soup.select("div.lister-item-content > p:nth-child(4)")

#### titles

In [ ]:
# Parse just the first page, for testing purposes

soup=BeautifulSoup(pages[0].content,"html.parser")

# Paste the Selector from the first movie title copied from Chrome Dev Tools

soup.select("#main > div > div.lister.list.detail.sub-list > div > div:nth-child(1) > div.lister-item-content > h3 > a")

# Trim the selection
soup.select("h3 > a")

#### synopsis

In [ ]:
# Paste the Selector from the first movie title copied from Chrome Dev Tools
soup.select("p:nth-child(4)")

### combine all the code 

There are many approaches to do this. The one we'll follow is: 

- Loop through the pages we collected, parse them ("create the soup") and store the parsed pages in a list. 

- For each parsed page, select the "blocks of HTML elements" that contain all the information of each movie (the title, the synopsis and other stuff). 

- For each one of the "blocks" we collected in the previous step: 

    - Get the movie titles and store them in a list 

    - Get the synopsis and store them in a list

In [ ]:
titles =[]
synopsis=[]
pages_parsed=[]

for i in range(len(pages)):
    pages_parsed.append(BeautifulSoup(pages[i].content,"html.parser"))
    movies_html=pages_parsed[i].select("div.lister-item-content")
    #for each movie, store title and synopsis into the lists
    for j in range(len(movies_html)):
        titles.append(movies_html[j].select("h3 > a")[0].get_text())
        synopsis.append(movies_html[j].select("p:nth-child(4)")[0].get_text().strip())

# check output
print(len(titles))
print(len(synopsis))


In [ ]:
len(pages)

In [ ]:
# check the output and identify any wrangling steps you missed
# next, you would create the data frame 

In [ ]:
titles[0:15]

In [ ]:
synopsis[0:5]

-----------

## 2nd example - Scraping presidents

Our objective is to create a dataframe with information about the presidents of the United States. To do this, we will go through 5 steps:

1. Scrape this [list of presidents of the United States](https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States).


In [ ]:
# 1. import libraries
import pandas as pd 


# 2. find url and store it in a variable
url="https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States"

# 3. download html with a get request
response=requests.get(url)
response.status_code

# 4.1. parse html (create the 'soup')
soup=BeautifulSoup(response.content,"html.parser")

# 4.2. check that the html code looks like it should
soup

2. Collect all the links to the Wikipedia page of each president.


In [ ]:
# this solution is not very elegant, but works. 
# The CSS selector we copied had an "nth-child" that we could iterate 
# to find presidents, but some elements were empty, so we concatenate 
# each new element with "+" instead of appending as usual:

presidents=[]

for i in range(84):
    presidents=presidents + soup.select("tbody > tr:nth-child(" +str(i)+ ") > td:nth-child(4) > b > a")

presidents



In [ ]:
# we can access the links searching for the attribute "href"
# in each element
presidents[45]["href"]
#https://en.wikipedia.org/wiki/Richard_Nixon

In [ ]:
# Now, we just assemble a new request to the link
url="https://en.wikipedia.org"+presidents[0]["href"]

# send request
response=requests.get(url)
response.status_code
# parse & store html
soup=BeautifulSoup(response.content, "html.parser")

In [ ]:
soup.find("table", {"class":"infobox vcard"})

3. Scrape the Wikipedia page of each president.


In this step we could very well store the whole wikipedia page for each president, or just the tiny, final pieces of information. Storing the boxes is a middle ground (we don't have too much noise but retain the flexibility of deciding later which specific elements to extract).

When sending multiple requests, remember to be respectful by spacing the requests a few seconds from each other. We will also ping the success code to monitor that everything is going well:

In [ ]:
# 2. find url and store it in a variable

presi_soups=[]

for presi in presidents:
    # send request
    url="https://en.wikipedia.org"+presi["href"]
    response=requests.get(url)
    print(presi.get_text(), response.status_code)
    # parse & store html
    soup=BeautifulSoup(response.content, "html.parser")
    presi_soups.append(soup.find("table", {"class":"infobox vcard"}))
    # respectful nap:
    wait_time=randint(1,2)
    print("I will sleep now for..."+str(wait_time)+"secs")
    sleep(wait_time)

4. Find and store information about each president.


We extracted the 'infoboxes': now it's time to extract specific information from them. First test what can we get from a single president and then assemble a loop for all of them.

Here, we will use [the string argument](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#the-string-argument) in the find function, since wikipedia tags and classes are not always helpful to locate. The string argument allows us to locate elements by its actual content.

In [ ]:
#Birthday
presi_soups[45].find("span",{"class":"bday"}).get_text()

#Political party
presi_soups([45].find("th",string="Political party").parent.find("a").get_text()

#Number of sons/daughters
len(presi_soups[45].find("th",string="Children").parent.find_all("li"))


In [ ]:
# collect with a loop - presidents name, their birthday , political party, no of children 
name=[]
dob=[]
party=[]
children=[]

for presi in presi_soups:
    name.append(presi.find("div",{"class":"fn"}).get_text())
    dob.append(presi.find("span",{"class":"bday"}).get_text())
    party.append(presi.find("th",string="Political party").parent.find("a").get_text())
    try:
        children.append(len(presi.find("th",string="Children").parent.find_all("li")))
    except:
        children.append(0)


5. Organize the information in a dataframe where we have each president as a row and each variable we collected as a column.

In [ ]:
presidents_data= pd.DataFrame({"name":name, "birthday":dob, "party":party, "noofchildren":children} )


In [ ]:
presidents_data